# Demographics and Body Measurements Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
!pip install pyreadstat
import pyreadstat #since the data files are .xpt files, this library is needed to import the table
import re
from nhanes_utils import to_snake_case, get_common_nan_ids, standardize_id_column, drop_rows_with_common_nan_ids

## Data Cleaning

Prior to initiating the data cleaning process, the required Python libraries are imported. Relevant NHANES datasets will be loaded sequentially at each step to maintain clarity and focus. This approach facilitates exploration of the data structure, variable types, and initial summaries, enabling a comprehensive understanding of the data's scope and quality.

### Demographics Data Cleaning

In [2]:
file_path = '2017-2020/2017-2020.P_DEMO.xpt'

df_demo, meta = pyreadstat.read_xport(file_path)
df_demo = standardize_id_column(df_demo)

In [3]:
df_demo.head(10)

,participant_id,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,DMDBORN4,...,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,WTINTPRP,WTMECPRP,SDMVPSU,SDMVSTRA,INDFMPIR
0,109263.0,66.0,2.0,1.0,2.0,NaN,5.0,6.0,2.0,1.0,...,2.0,NaN,NaN,NaN,NaN,7891.762435,8951.815567,3.0,156.0,4.66
1,109264.0,66.0,2.0,2.0,13.0,NaN,1.0,1.0,2.0,1.0,...,2.0,1.0,2.0,2.0,1.0,11689.747264,12271.157043,1.0,155.0,0.83
2,109265.0,66.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,1.0,...,2.0,NaN,NaN,NaN,NaN,16273.825939,16658.764203,1.0,157.0,3.06
3,109266.0,66.0,2.0,2.0,29.0,NaN,5.0,6.0,2.0,2.0,...,2.0,1.0,2.0,2.0,1.0,7825.646112,8154.968193,2.0,168.0,5.00
4,109267.0,66.0,1.0,2.0,21.0,NaN,2.0,2.0,NaN,2.0,...,2.0,NaN,NaN,NaN,NaN,26379.991724,0.000000,1.0,156.0,5.00
5,109268.0,66.0,1.0,2.0,18.0,NaN,3.0,3.0,NaN,1.0,...,2.0,NaN,NaN,NaN,NaN,19639.221008,0.000000,1.0,155.0,1.66
6,109269.0,66.0,2.0,1.0,2.0,NaN,2.0,2.0,1.0,1.0,...,2.0,NaN,NaN,NaN,NaN,5906.250521,6848.271782,2.0,152.0,0.96
7,109270.0,66.0,2.0,2.0,11.0,NaN,4.0,4.0,1.0,1.0,...,2.0,1.0,2.0,2.0,NaN,4613.057696,4886.930378,1.0,150.0,1.88
8,109271.0,66.0,2.0,1.0,49.0,NaN,3.0,3.0,2.0,1.0,...,2.0,1.0,2.0,2.0,1.0,8481.589837,8658.732873,1.0,167.0,NaN
9,109272.0,66.0,2.0,1.0,0.0,3.0,1.0,1.0,2.0,1.0,...,2.0,NaN,NaN,NaN,NaN,7037.380216,7872.776233,1.0,155.0,0.73


Many of the original column names in the dataset are not immediately intuitive. To improve readability and facilitate analysis, we will first list all column names and then rename them with more straightforward, descriptive labels. The original column descriptions can be referenced here: NHANES Demographics Codebook.

Additionally, based on the project’s scope, several columns that are not relevant to our analysis will be dropped to streamline the dataset.

In [4]:
df_demo.columns

Index(['participant_id', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR',
       'RIDAGEMN', 'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'DMDBORN4', 'DMDYRUSZ',
       'DMDEDUC2', 'DMDMARTZ', 'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP',
       'FIALANG', 'FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP',
       'AIALANGA', 'WTINTPRP', 'WTMECPRP', 'SDMVPSU', 'SDMVSTRA', 'INDFMPIR'],
      dtype='object')

In [5]:
df_demo.drop(['SDDSRVYR','RIDAGEMN','RIDRETH1','RIDEXMON','DMDBORN4','DMDYRUSZ','RIDEXPRG','SIALANG', 
         'SIAPROXY', 'SIAINTRP','FIALANG','FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 
         'MIAINTRP', 'AIALANGA','WTINTPRP', 'WTMECPRP', 'SDMVPSU', 'SDMVSTRA'],
        axis=1,inplace=True, errors='ignore')

In [6]:
df_demo.columns

Index(['participant_id', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3',
       'DMDEDUC2', 'DMDMARTZ', 'INDFMPIR'],
      dtype='object')

In [7]:
df_demo = df_demo.rename(columns={
    'RIAGENDR': 'gender',
    'RIDAGEYR' : 'age_year',
    'RIDRETH3' : 'race',
    'DMDEDUC2' : 'education_level', 
    'DMDMARTZ' : 'marital_status',
    'INDFMPIR' : 'family_income_poverty',
    'RIDSTATR' : 'interview_exam_status'
})

In [8]:
df_demo.head(10)

,participant_id,interview_exam_status,gender,age_year,race,education_level,marital_status,family_income_poverty
0,109263.0,2.0,1.0,2.0,6.0,NaN,NaN,4.66
1,109264.0,2.0,2.0,13.0,1.0,NaN,NaN,0.83
2,109265.0,2.0,1.0,2.0,3.0,NaN,NaN,3.06
3,109266.0,2.0,2.0,29.0,6.0,5.0,3.0,5.00
4,109267.0,1.0,2.0,21.0,2.0,4.0,3.0,5.00
5,109268.0,1.0,2.0,18.0,3.0,NaN,NaN,1.66
6,109269.0,2.0,1.0,2.0,2.0,NaN,NaN,0.96
7,109270.0,2.0,2.0,11.0,4.0,NaN,NaN,1.88
8,109271.0,2.0,1.0,49.0,3.0,2.0,3.0,NaN
9,109272.0,2.0,1.0,0.0,1.0,NaN,NaN,0.73


In [9]:
df_demo.dtypes

participant_id           float64
interview_exam_status    float64
gender                   float64
age_year                 float64
race                     float64
education_level          float64
marital_status           float64
family_income_poverty    float64
dtype: object

In [10]:
df_demo.shape

(15560, 8)

In [11]:
df_demo.isnull().sum()

participant_id              0
interview_exam_status       0
gender                      0
age_year                    0
race                        0
education_level          6328
marital_status           6328
family_income_poverty    2201
dtype: int64

In [12]:
df_demo.describe()

,participant_id,interview_exam_status,gender,age_year,race,education_level,marital_status,family_income_poverty
count,15560.00000,15560.000000,15560.000000,15560.000000,15560.000000,9232.000000,9232.000000,13359.000000
mean,117042.50000,1.919023,1.503792,33.742481,3.486118,3.551993,1.708622,2.405937
std,4491.92943,0.272808,0.500002,25.320532,1.622734,1.214109,2.755878,1.634346
min,109263.00000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,113152.75000,2.000000,1.000000,10.000000,3.000000,3.000000,1.000000,1.020000
50%,117042.50000,2.000000,2.000000,30.000000,3.000000,4.000000,1.000000,1.960000
75%,120932.25000,2.000000,2.000000,56.000000,4.000000,4.000000,2.000000,3.880000
max,124822.00000,2.000000,2.000000,80.000000,7.000000,9.000000,99.000000,5.000000


Based on the NHANES data documentation, the variable RIDSTATR (renamed as interview_exam_status) indicates whether a participant completed only the interview (1) or both the interview and physical examination (2). Since this project focuses on physical and medical examination data, participants with a value of 1 will be excluded. After filtering out these participants, the interview_exam_status column will be dropped, as it will no longer provide useful information.

In [13]:
df_demo = df_demo[df_demo['interview_exam_status'] != 1]

In [14]:
df_demo.head(10)

,participant_id,interview_exam_status,gender,age_year,race,education_level,marital_status,family_income_poverty
0,109263.0,2.0,1.0,2.0,6.0,NaN,NaN,4.66
1,109264.0,2.0,2.0,13.0,1.0,NaN,NaN,0.83
2,109265.0,2.0,1.0,2.0,3.0,NaN,NaN,3.06
3,109266.0,2.0,2.0,29.0,6.0,5.0,3.0,5.00
6,109269.0,2.0,1.0,2.0,2.0,NaN,NaN,0.96
7,109270.0,2.0,2.0,11.0,4.0,NaN,NaN,1.88
8,109271.0,2.0,1.0,49.0,3.0,2.0,3.0,NaN
9,109272.0,2.0,1.0,0.0,1.0,NaN,NaN,0.73
10,109273.0,2.0,1.0,36.0,3.0,4.0,3.0,0.83
11,109274.0,2.0,1.0,68.0,7.0,4.0,3.0,1.20


In [15]:
df_demo.drop(['interview_exam_status'], axis = 1, inplace = True)

In [16]:
df_demo.shape

(14300, 7)

After the initial cleaning process, the resulting dataset contains approximately 143,000 rows and 7 columns.

To maintain consistency with the original NHANES coding and to support traceability, the decision was made to retain the raw categorical values for columns containing encoded data (such as gender, race/ethnicity, and education level). These encoded values follow NHANES's standard coding conventions and will be referenced using corresponding lookup tables or detailed clarifications provided in the README.

In [17]:
df_demo.head(10)

,participant_id,gender,age_year,race,education_level,marital_status,family_income_poverty
0,109263.0,1.0,2.0,6.0,NaN,NaN,4.66
1,109264.0,2.0,13.0,1.0,NaN,NaN,0.83
2,109265.0,1.0,2.0,3.0,NaN,NaN,3.06
3,109266.0,2.0,29.0,6.0,5.0,3.0,5.00
6,109269.0,1.0,2.0,2.0,NaN,NaN,0.96
7,109270.0,2.0,11.0,4.0,NaN,NaN,1.88
8,109271.0,1.0,49.0,3.0,2.0,3.0,NaN
9,109272.0,1.0,0.0,1.0,NaN,NaN,0.73
10,109273.0,1.0,36.0,3.0,4.0,3.0,0.83
11,109274.0,1.0,68.0,7.0,4.0,3.0,1.20


At this stage, missing values (NaN) in other columns are retained to maximize the number of participants included in the dataset. This approach preserves potential correlations with other lab and physical examination variables. The cleaned dataframe has been saved as a CSV file for subsequent import into a relational database.

In [18]:
df_demo.to_csv('cleaned_demographics.csv', index=False)

### Body Measures Data Cleaning

In [19]:
file_path = '2017-2020/2017-2020.P_BMX.xpt'

df_bmx, meta = pyreadstat.read_xport(file_path)
df_bmx = standardize_id_column(df_bmx)

In [20]:
df_bmx.head(10)

,participant_id,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
0,109263.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,1.0,42.2,NaN,NaN,NaN,NaN,NaN,154.7,NaN,...,36.3,NaN,33.8,NaN,22.7,NaN,63.8,NaN,85.0,NaN
2,109265.0,1.0,12.0,NaN,91.6,NaN,NaN,NaN,89.3,NaN,...,NaN,NaN,18.6,NaN,14.8,NaN,41.2,NaN,NaN,NaN
3,109266.0,1.0,97.1,NaN,NaN,NaN,NaN,NaN,160.2,NaN,...,40.8,NaN,34.7,NaN,35.8,NaN,117.9,NaN,126.1,NaN
4,109269.0,3.0,13.6,NaN,90.9,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
5,109270.0,1.0,75.3,NaN,NaN,NaN,NaN,NaN,156.0,NaN,...,42.6,NaN,36.1,NaN,31.0,NaN,91.4,NaN,NaN,NaN
6,109271.0,1.0,98.8,NaN,NaN,NaN,NaN,NaN,182.3,NaN,...,40.1,NaN,42.0,NaN,38.2,NaN,120.4,NaN,108.2,NaN
7,109272.0,1.0,7.1,NaN,63.6,NaN,41.3,NaN,NaN,NaN,...,NaN,NaN,13.0,NaN,15.5,NaN,NaN,NaN,NaN,NaN
8,109273.0,1.0,74.3,NaN,NaN,NaN,NaN,NaN,184.2,NaN,...,41.0,NaN,41.1,NaN,30.2,NaN,86.8,NaN,94.5,NaN
9,109274.0,1.0,103.7,NaN,NaN,NaN,NaN,NaN,185.3,NaN,...,44.0,NaN,47.0,NaN,32.0,NaN,109.6,NaN,107.8,NaN


In [21]:
df_bmx.shape

(14300, 22)

In [22]:
df_bmx.isnull().sum()

participant_id        0
BMDSTATS              0
BMXWT               225
BMIWT             13712
BMXRECUM          12830
BMIRECUM          14257
BMXHEAD           13990
BMIHEAD           14300
BMXHT              1143
BMIHT             14129
BMXBMI             1163
BMDBMIC            9551
BMXLEG             3316
BMILEG            13812
BMXARML             810
BMIARML           13813
BMXARMC             816
BMIARMC           13807
BMXWAIST           1726
BMIWAIST          13683
BMXHIP             4438
BMIHIP            13924
dtype: int64

Looking at the document associated with this dataset, each of the column and their contents were assessed. There are participants without body measurement data, which will be excluded. 

In [23]:
df_bmx = df_bmx[df_bmx['BMDSTATS'] != 4]

In [24]:
df_bmx.shape

(14107, 22)

The head circumference is measured for children 0 years - 6 months old. Since this value is obtained only for a small portion of the participant population, the decision was made to drop that column. 

There are some columns with comments to explain some of the missing data and the decision was made to keep those. The rest of the columns will be renamed to be more intuitive.

In [25]:
df_bmx.drop(['BMDSTATS','BMXHEAD','BMIHEAD'], axis = 1, inplace = True)

In [26]:
df_bmx.columns

Index(['participant_id', 'BMXWT', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHT',
       'BMIHT', 'BMXBMI', 'BMDBMIC', 'BMXLEG', 'BMILEG', 'BMXARML', 'BMIARML',
       'BMXARMC', 'BMIARMC', 'BMXWAIST', 'BMIWAIST', 'BMXHIP', 'BMIHIP'],
      dtype='object')

In [27]:
df_bmx = df_bmx.rename(columns={
    'BMXWT':'weight_kg',
    'BMIWT': 'weight_comment',
    'BMXRECUM': 'recumbent_length_cm',
    'BMIRECUM': 'recumbent_length_comment',
    'BMXHT': 'standing_height', 
    'BMIHT': 'standing_height_comment',
    'BMXBMI': 'bmi',
    'BMDBMIC': 'bmi_category_child',
    'BMXLEG': 'upper_leg_cm',
    'BMILEG': 'upper_leg_comment',
    'BMXARML': 'upper_arm_cm',
    'BMIARML': 'upper_arm_comment',
    'BMXARMC': 'arm_circumference_cm',
    'BMIARMC': 'arm_circ_comment',
    'BMXWAIST': 'waist_circ_cm',
    'BMIWAIST': 'waist_circ_comment', 
    'BMXHIP': 'hip_circ_cm', 
    'BMIHIP': 'hip_circ_comment'
})

In [28]:
df_bmx.head(10)

,participant_id,weight_kg,weight_comment,recumbent_length_cm,recumbent_length_comment,standing_height,standing_height_comment,bmi,bmi_category_child,upper_leg_cm,upper_leg_comment,upper_arm_cm,upper_arm_comment,arm_circumference_cm,arm_circ_comment,waist_circ_cm,waist_circ_comment,hip_circ_cm,hip_circ_comment
1,109264.0,42.2,NaN,NaN,NaN,154.7,NaN,17.6,2.0,36.3,NaN,33.8,NaN,22.7,NaN,63.8,NaN,85.0,NaN
2,109265.0,12.0,NaN,91.6,NaN,89.3,NaN,15.0,2.0,NaN,NaN,18.6,NaN,14.8,NaN,41.2,NaN,NaN,NaN
3,109266.0,97.1,NaN,NaN,NaN,160.2,NaN,37.8,NaN,40.8,NaN,34.7,NaN,35.8,NaN,117.9,NaN,126.1,NaN
4,109269.0,13.6,NaN,90.9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
5,109270.0,75.3,NaN,NaN,NaN,156.0,NaN,30.9,4.0,42.6,NaN,36.1,NaN,31.0,NaN,91.4,NaN,NaN,NaN
6,109271.0,98.8,NaN,NaN,NaN,182.3,NaN,29.7,NaN,40.1,NaN,42.0,NaN,38.2,NaN,120.4,NaN,108.2,NaN
7,109272.0,7.1,NaN,63.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,15.5,NaN,NaN,NaN,NaN,NaN
8,109273.0,74.3,NaN,NaN,NaN,184.2,NaN,21.9,NaN,41.0,NaN,41.1,NaN,30.2,NaN,86.8,NaN,94.5,NaN
9,109274.0,103.7,NaN,NaN,NaN,185.3,NaN,30.2,NaN,44.0,NaN,47.0,NaN,32.0,NaN,109.6,NaN,107.8,NaN
10,109275.0,20.9,NaN,NaN,NaN,120.4,NaN,14.4,2.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN


With the comment sections, most of the sections are marked "Could not obtain". The comment sections that only have such notation was removed, as the lack of value is self evident with missing values (NaN).

In [29]:
df_bmx.drop(['recumbent_length_comment','upper_leg_comment','upper_arm_comment',
         'arm_circ_comment','waist_circ_comment','hip_circ_comment'], axis = 1, inplace = True)

In [30]:
df_bmx.head(10)

,participant_id,weight_kg,weight_comment,recumbent_length_cm,standing_height,standing_height_comment,bmi,bmi_category_child,upper_leg_cm,upper_arm_cm,arm_circumference_cm,waist_circ_cm,hip_circ_cm
1,109264.0,42.2,NaN,NaN,154.7,NaN,17.6,2.0,36.3,33.8,22.7,63.8,85.0
2,109265.0,12.0,NaN,91.6,89.3,NaN,15.0,2.0,NaN,18.6,14.8,41.2,NaN
3,109266.0,97.1,NaN,NaN,160.2,NaN,37.8,NaN,40.8,34.7,35.8,117.9,126.1
4,109269.0,13.6,NaN,90.9,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,109270.0,75.3,NaN,NaN,156.0,NaN,30.9,4.0,42.6,36.1,31.0,91.4,NaN
6,109271.0,98.8,NaN,NaN,182.3,NaN,29.7,NaN,40.1,42.0,38.2,120.4,108.2
7,109272.0,7.1,NaN,63.6,NaN,NaN,NaN,NaN,NaN,13.0,15.5,NaN,NaN
8,109273.0,74.3,NaN,NaN,184.2,NaN,21.9,NaN,41.0,41.1,30.2,86.8,94.5
9,109274.0,103.7,NaN,NaN,185.3,NaN,30.2,NaN,44.0,47.0,32.0,109.6,107.8
10,109275.0,20.9,NaN,NaN,120.4,NaN,14.4,2.0,NaN,NaN,NaN,NaN,NaN


In [31]:
df_bmx.shape

(14107, 13)

The comment sections that are kept have values more than just "Not Obtained". These will be interpreted with appropriate comment and translated into string values

In [32]:
df_bmx.to_csv('cleaned_bodymeasures.csv', index=False)

The same cleaning and preprocessing steps will be applied to the other tables and datasets to ensure consistency and maintain data quality for the forthcoming analysis.